In [1]:
import pandas as pd
import numpy as np
import spacy
import preprocessing
from csv import reader 
import tfidf
import scipy


pd.set_option('max_colwidth', 5000)



In [2]:
df = pd.read_csv('DB/Original_data.csv', index_col=[0])
df = df.sample(frac=0.2)
#Check if there are null values 
df.isna().sum()

id         0
title      0
body       0
tags       0
answers    0
score      0
dtype: int64

In [3]:
#create column with answers aggregate by title of the questions
df2 = df 
func = lambda x: "\n".join(x)
df2 = df2.groupby('id')["answers"].agg([("answers",func)])



In [4]:
# concat answers for each post beacause there are a record with question duplicated for each answer to that question 


grouped = df.groupby(['id','title', 'body','tags'],as_index=False).agg("sum","score")

grouped_df = pd.DataFrame(grouped)
grouped_df = pd.merge(grouped_df, df2, left_on='id', right_on='id', how='left')
grouped_df



id  \
0           742   
1           773   
2          2933   
3          5102   
4          5136   
...         ...   
87089  69058561   
87090  69059121   
87091  69059605   
87092  69059759   
87093  69059948   

                                                                                title  \
0                                                               Class views in Django   
1                                                   How do I use itertools.groupby()?   
2                    Create a directly-executable cross-platform GUI app using Python   
3                             How do you set up Python scripts to work in Apache 2.0?   
4                    Does anyone have experience creating a shared library in MATLAB?   
...                                                                               ...   
87089                            Getting an error while working on a selenium project   
87090                                 Can you draw a normal curve on seaborn displot?   
87091                       How to check if a column of DataFrame contain float type?   
87092                         Replacing one line of a file with contents of two files   
87093  Check if elements in numpy array are between columns of a 2-column numpy array   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      body  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           <p><a href="http://www.djangoproject.com/" rel="noreferrer">Django</a> view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach.</p>\n\n<p>For example, I have a page that displays a user. This page is very similar to page that displays a group, but it's still not so similar to just use another data model. Group also has members etc...</p>\n\n<p>One way would be to point views to class methods and then extend that class. Has anyone tried this approach or has any other idea? </p>   
1                              

In [5]:
#Preprocessing Part
#Answers
#Removing tags


answers = grouped_df["answers"]
preprocessing.remove_tags(answers)



#Clearing text 
answers_processed = answers.apply(lambda x: preprocessing.clear_text(x))


NameError: name 'final_text' is not defined

In [ ]:
answers_processed.head()
answers_processed.isna().sum() 
grouped_df['answers_processed'] = answers_processed

In [ ]:
#Questions
#Merge title with body 
questions = grouped_df["body"]
preprocessing.remove_tags(questions)
questions

In [ ]:
#Clearing text 
questions_processed = questions.apply(lambda x: preprocessing.clear_text(x))
grouped_df['questions_processed'] = questions_processed
questions_processed

In [ ]:
#Create a column only for the processed title of the questions 
processed_title = grouped_df.title.apply(lambda x: preprocessing.clear_text(x))
grouped_df['processed_title'] = processed_title
processed_title 

In [ ]:
#Create a column to add in db with question title, question body and all the answers processed, to train later the W2V embeddings
#post_corpus = processed_title + '\n '+ questions_processed + '\n ' + answers_processed
grouped_df.drop("answers", axis=1, inplace=True)
grouped_df.drop("body", axis=1, inplace=True)
#grouped_df["post_corpus"] = post_corpus
grouped_df["questions"] = questions
grouped_df

In [ ]:
#Filter out the tags, selecting only the 20 most common for better processing 
# Convert raw text data of tags into lists
grouped_df["tags"] = grouped_df["tags"].apply(lambda x: x.split('|'))   

# Make a dictionary to count the frequencies for all tags
tag_freq_dict = {}

#i = 0 

for tags in grouped_df["tags"]:
    for tag in tags:
        #Remove tags python, python2.7 e python3 for further processing 
        if "python" not in tag :
            
            if tag not in tag_freq_dict:
                tag_freq_dict[tag] = 0
            else:
                tag_freq_dict[tag] += 1
        else:
            tags.remove(tag)
    '''
    #If a record has less than 3 tags eliminate it for better further processing (training of fastText)
    if len(tags) < 3:
        grouped_df.drop(i)
    i = i +1
    '''
grouped_df["tags"]


In [ ]:
#Selecting the most common number of tags in our database 
import heapq
most_common_tags = heapq.nlargest(30, tag_freq_dict, key=tag_freq_dict.get)
most_common_tags

In [ ]:
#Select only the data with at least one of the most common tags 
final_indices = []
for i,tags in enumerate(grouped_df["tags"].values.tolist()):
    if len(set(tags).intersection(set(most_common_tags)))>0:   # The minimum length for common tags should be 1
        final_indices.append(i)

final_data = grouped_df.iloc[final_indices]

final_data 

In [ ]:
# Normalize numeric data for the scores
final_data['score'] = (final_data['score'] - final_data['score'].mean()) / (final_data['score'].max() - final_data['score'].min())

In [ ]:
# Combine the lists back into text data
final_data['tags'] = final_data['tags'].apply(lambda x: '|'.join(x))

final_data

In [ ]:
#Check if the final data has some null values 
final_data.isna().sum()

final_data = final_data.dropna()
#final_data = final_data[final_data['post_corpus'].notna()]
final_data = final_data[final_data['processed_title'].notna()]
final_data 

In [ ]:
#Create matrix for tdfidf for further processing 
corpus = final_data['processed_title'] + final_data['questions_processed'] + final_data['answers_processed'] 
matrix = tfidf.create_matrix(corpus)
scipy.sparse.save_npz('DB/tfidf_stack_matrix.npz', matrix)

In [ ]:
# Save the data
final_data.to_csv('DB/Preprocessed_data.csv', index=False)